In [1]:
import pandas as pd
data = pd.read_csv('../datasets/mtsamples.csv')
data.head()
data['transcription'][15]

"HISTORY OF PRESENT ILLNESS: ,I have seen ABC today for her preoperative visit for weight management.  I have explained to her the need for Optifast for weight loss prior to these procedures to make it safer because of the large size of her liver.  She understands this.,IMPRESSION/PLAN:, We are going to put her on two weeks of Optifast at around 900 calories.  I have also explained the risks and potential complications of laparoscopic cholecystectomy to her in detail including bleeding, infection, deep venous thrombosis, pulmonary embolism, injury to the small intestine, stomach, liver, leak from the cystic duct, common bile duct, and possible need for ERCP and further surgery.  This surgery is going to be planned for October 6.  This is for cholelithiasis prior to her Lap-Banding procedure.,I have also reviewed with her the risks and potential complications of laparoscopic gastric banding including bleeding, infection, deep venous thrombosis, pulmonary embolism, slippage of the band, 

## keywords listing

In [2]:
keywords = data['keywords']
print(keywords)

#! we have bunch of nan values, that may hamper our model prediction

0       allergy / immunology, allergic rhinitis, aller...
1       bariatrics, laparoscopic gastric bypass, weigh...
2       bariatrics, laparoscopic gastric bypass, heart...
3       cardiovascular / pulmonary, 2-d m-mode, dopple...
4       cardiovascular / pulmonary, 2-d, doppler, echo...
                              ...                        
4994                                                  NaN
4995    allergy / immunology, mucous membranes, conjun...
4996                                                  NaN
4997                                                  NaN
4998                                                  NaN
Name: keywords, Length: 4999, dtype: object


In [3]:
# Split the strings in the 'keywords' column by comma and whitespace             
keywords_list = keywords.str.split(', ').explode()

# Convert the resulting Series to a 1D array
keywords_array = keywords_list.to_numpy()

# Now, keywords_array is a 1D NumPy array containing all the symptoms
print(keywords_array)

['allergy / immunology' 'allergic rhinitis' 'allergies' ... nan nan nan]


## Identify Symptoms

### This technique is quite simple
### We have just extract the Nouns and Pronouns. Then match those with existing datasets Keyword array.

In [4]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract symptoms from a patient description
def entity_extraction(patient_description):
    
  """Extracts symptoms from a patient description using spaCy."""

  # Create a spaCy document from the patient description
  doc = nlp(patient_description)
      
  noun_phrases = [chunk for chunk in doc.noun_chunks if chunk.root.pos_ in ["NOUN", "PROPN"]]              

  # Extract the noun phrase text
  symptoms = [chunk.text for chunk in noun_phrases]

  # Return the list of symptoms
  return symptoms

# Example patient description
patient_description = """
I've been feeling really unwell lately. It started a few days ago with a persistent headache that just won't go away. I've also noticed a high fever, which has been keeping me up at night. My throat feels sore, and I've been coughing a lot. I've lost my appetite, and I feel weak and fatigued all the time. In addition to all of this, I've been experiencing some nausea and occasional vomiting. It's been quite challenging to focus or go about my daily activities with all these symptoms.
"""

# for row in data['transcription']:
#   print(row)
#   print ("=====")

# Extract symptoms from the patient description
symptoms = entity_extraction(patient_description)

# Print the extracted symptoms
print(symptoms)

['a persistent headache', 'a high fever', 'night', 'My throat', 'a lot', 'my appetite', 'addition', 'some nausea', 'occasional vomiting', 'my daily activities', 'all these symptoms']


In [5]:
symptoms_arr = [str(keyword) for keyword in keywords_array if isinstance(keyword, str)]
pruned_array = [item for item in symptoms if item in symptoms_arr]
print(pruned_array)

[]
